In [8]:
import re
import shutil
import time
from pathlib import Path

from selenium.webdriver import Edge, EdgeOptions
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait


In [9]:
def fun_页面标题等待(title: str, driver: Edge):
    return WebDriverWait(driver=driver, timeout=10, poll_frequency=1).until(
        expected_conditions.title_contains(title)
    )


def fun_js点击(selector: str, driver: Edge):
    driver.execute_script(f'document.querySelector("{selector}").click()')


def fun_获取数字(stem: str) -> int:
    num_list = re.findall(r"\d+", stem)
    if len(num_list) > 0:
        return int("".join(num_list))
    return 0

In [10]:
options = EdgeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

driver = Edge(options=options)
driver.set_window_rect(x=1274, y=0, width=1292, height=1398)

driver.get(
    "https://havanalogin.taobao.com/mini_login.htm?lang=zh_CN&appName=taobao&appEntrance=qianniu_pc_web&styleType=vertical&bizParams=&notLoadSsoView=true&notKeepLogin=false&isMobile=false&cssUrl=https://g.alicdn.com/qn/qn-login-iframe-css/0.0.8/qn-login-iframe.css&returnUrl=https://myseller.taobao.com/home.htm&rnd=0.17371788128615084"
)
fun_页面标题等待("登录", driver)

SessionNotCreatedException: Message: session not created: Microsoft Edge failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from msedge location C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe is no longer running, so msedgedriver is assuming that msedge has crashed.)
Stacktrace:
	GetHandleVerifier [0x00007FF76A8A3A22+10594]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF76A840449+323353]
	(No symbol) [0x00007FF76A65F16E]
	(No symbol) [0x00007FF76A692DD2]
	(No symbol) [0x00007FF76A68DF68]
	(No symbol) [0x00007FF76A6D19CD]
	(No symbol) [0x00007FF76A6C77E3]
	(No symbol) [0x00007FF76A69B3EC]
	(No symbol) [0x00007FF76A69A8CD]
	(No symbol) [0x00007FF76A69AFB1]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF76AA2E4C4+918756]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF76A7CC9BF+54575]
	Microsoft::Applications::Events::EventProperty::~Even

In [998]:
# 输入账号密码进行登录，会自动跳入到后台
driver.find_element(By.CSS_SELECTOR, "#fm-login-id").send_keys(input("用户名"))
driver.find_element(By.CSS_SELECTOR, "#fm-login-password").send_keys(input("密码"))
driver.find_element(By.CSS_SELECTOR, "#login-form > div.fm-btn > button").click()
fun_页面标题等待("千牛商家工作台", driver)

True

In [1004]:
# ---------------- 操作开始的地方 ----------------


for material_path in Path(R"C:\Users\wuweihua\Desktop\UPLOAD").iterdir():
    # 打开发布页面
    driver.get(
        "https://item.upload.taobao.com/sell/v2/publish.htm?catId=201160807&smartRouter=true&keyProps=%7B%7D&newRouter=1&paramCacheId=merge_router_cache_389353239_1694673279246_988&x-gpf-submit-trace-id=213e259e16946732792022149e0968"
    )
    fun_页面标题等待("商品发布", driver)

    # 标题
    ActionChains(driver).send_keys_to_element(
        driver.find_element(
            By.CSS_SELECTOR,
            "#sell-field-title > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > span > span > span > span > input",
        ),
        material_path.stem,
    ).perform()

    # 价格
    ActionChains(driver).send_keys_to_element(
        driver.find_element(
            By.CSS_SELECTOR,
            "#sell-field-price > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
        ),
        "3.8",
    ).perform()

    # 库存
    driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-quantity > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
    ).send_keys(Keys.CONTROL, "a")
    time.sleep(0.5)
    driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-quantity > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
    ).send_keys(Keys.DELETE)
    time.sleep(0.5)
    driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-quantity > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
    ).send_keys("888888")
    time.sleep(1)

    # 商家编码
    driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-outerId > div.sell-component-info-wrapper-component-wrap > div > div > span > span > span > span > input",
    ).send_keys(material_path.stem)
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)

    # 点击上传主图
    driver.find_element(
        By.CSS_SELECTOR, "#struct-mainImagesGroup > div > div > div > div > div > div"
    ).click()

    # 切换到上传图片frame
    driver.switch_to.parent_frame()
    driver.switch_to.frame(
        driver.find_element(
            By.CSS_SELECTOR,
            "body > div.next-overlay-wrapper.v2.opened > div > div > div > iframe",
        )
    )
    time.sleep(2)

    # 点击ST文件夹
    driver.find_element(
        By.CSS_SELECTOR,
        "#sucai-tu-selector-container > div.Home_body__P3Ncj.Home_withoutHeaderBody__DxGAT > div.PicGroupList_aside__fp5JR > div > div > div > div > ul > li:nth-child(2) > div",
    ).click()
    time.sleep(2)

    # 点击暴富首图
    driver.find_element(
        By.CSS_SELECTOR,
        "#sucai_tu_selector_scrollMain > div > div > label > div.PicList_pic_imgBox__c0HXw > img",
    ).click()
    time.sleep(2)

    # 点击确定
    driver.find_element(
        By.CSS_SELECTOR,
        "#sucai-tu-selector-container > div.Footer_contain__g740- > div:nth-child(2) > button.next-btn.next-medium.next-btn-primary.Footer_selectOk__nEl3N",
    ).click()
    time.sleep(1)

    # 切回到主框架
    driver.switch_to.parent_frame()
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)

    # 详情图片上传按钮
    js_text = 'document.querySelector("#sell-field-descRepublicOfSell > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > div.editor--GiZhF > div > div.editPanel--nAdw5.lite-editor-panel > div.editHeader--HV9VI.sidePadding--Zi2zU > div.leftPart--RCPnT > div:nth-child(1)").click()'
    driver.execute_script(js_text)
    time.sleep(2)

    # 进入到图片上传frame
    driver.switch_to.frame(
        driver.find_element(
            By.CSS_SELECTOR,
            "body > div.ln-overlay-wrapper.opened > div.ln-overlay-inner.mediaDialog--EM7Hx.ck-dialog.overlay--Bwotz > div > iframe",
        )
    )

    # 点击上传图片按钮
    driver.find_element(
        By.CSS_SELECTOR,
        "#container > div > div.hasEms.container > div.main > div > div.opt-body > div.list > div.search-bar > div.right-btn.btn",
    ).click()
    time.sleep(2)

    # 构建图片列表
    pic_list: list[Path] = []
    for in_file in (material_path).iterdir():
        if in_file.is_file() and in_file.suffix.lower() == ".jpg":
            pic_list.append(in_file)

            if len(pic_list) == 20:
                break

    pic_list.sort(key=lambda k: fun_获取数字(k.stem))

    # 上传
    pic_text = "\n".join([pic.as_posix() for pic in pic_list])
    driver.find_element(
        By.XPATH,
        "/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div[3]/input",
    ).send_keys(pic_text)

    # 等待时间
    time.sleep(len(pic_list) / 1.5)

    # 选择图片
    pic_item_list = driver.find_elements(By.CSS_SELECTOR, "#items > div")[
        : len(pic_list)
    ]

    for pic_item in pic_list:
        for item in pic_item_list:
            name = item.find_element(By.CSS_SELECTOR, ".name").text
            if name == pic_item.name:
                item.location_once_scrolled_into_view
                item.find_element(By.CSS_SELECTOR, "img").click()
                time.sleep(0.5)

    # 点击确认
    driver.find_element(
        By.CSS_SELECTOR,
        "#container > div > div.hasEms.container > div.main > div > div.opt-footer > div.btn.btn-blue",
    ).click()
    time.sleep(3)

    # 切回到主框架
    driver.switch_to.default_content()
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)

    # 放入仓库
    fun_js点击(
        selector="#sell-field-startTime > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > span > span:nth-child(3) > label",
        driver=driver,
    )
    time.sleep(1)

    # 提交宝贝
    fun_js点击(selector="#struct-buttons > button", driver=driver)
    time.sleep(1)

    # 删除文件夹
    shutil.rmtree(material_path)

    time.sleep(5)